In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
import os

/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- 1. Konfigurace ---
# Cesta k vašemu lokálně uloženému modelu
MODEL_PATH = "/home/luk-hrd/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/2dcff66eac0c01dc50e4c41eea959968232187fe"

# Název datasetu na Hugging Face Hubu
DATASET_NAME = "BoltzmannEntropy/QuantumLLMInstruct"

# Název pro váš nový, fine-tunovaný model (pro výstupní adresář)
NEW_MODEL_NAME = "mistral-7b-quantum-instruct-v1"

In [3]:
# --- 2. Načtení a příprava datasetu ---

# Načteme pouze trénovací split datasetu
dataset = load_dataset(DATASET_NAME, split="train")

# Vylepšená formátovací funkce, která přidává kontext z dalších sloupců
def format_prompt_with_context(sample):
    # Bezpečné načtení hodnot, pokud by chyběly
    main_domain = sample.get('main_domain', 'quantum computing')
    sub_domain = sample.get('sub_domain', 'a specific topic')
    problem = sample['problem']
    solution = sample['solution']
    
    # Vytvoření bohatší a specifičtější instrukce
    instruction = (
        f"You are an expert in '{main_domain}', specifically focusing on '{sub_domain}'. "
        f"Provide a detailed, step-by-step solution to the following problem:\n\n"
        f"### Problem:\n{problem}"
    )
    
    # Sestavení finálního textu pro trénink
    return f"[INST] {instruction} [/INST] {solution}"

print("Příklad naformátovaného promptu:")
print(format_prompt_with_context(dataset[0]))


Příklad naformátovaného promptu:
[INST] You are an expert in 'Quantum Thermodynamics', specifically focusing on 'QUANTUM THERMODYNAMICS'. Provide a detailed, step-by-step solution to the following problem:

### Problem:
Consider a two-qubit system initially prepared in the state \( |\psi_0\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}} \). The Hamiltonian of the system is given by \( H = \sigma_1^z \sigma_2^z \), where \( \sigma_i^z \) is the Pauli z-operator for qubit \( i \). The system is then allowed to evolve under the dynamics of the Hamiltonian for a time \( t = \pi \). Calculate the von Neumann entropy of the system at \( t = \pi \). [/INST] To solve this problem, we need to follow these steps:

1. **Express the initial state in matrix form:**
   The initial state \( |\psi_0\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}} \) can be written in matrix form as:
   \[
   |\psi_0\rangle = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 \\ 0 \\ 0 \\ 1 \end{pmatrix}
   \]

2. **Define the P

In [4]:
# --- 3. Konfigurace a načtení modelu a tokenizeru ---

# Konfigurace 4-bitové kvantizace pro úsporu VRAM
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Používá pokročilejší typ kvantizace "NormalFloat 4"
    bnb_4bit_compute_dtype=torch.bfloat16,  # Pro výpočty použije bfloat16 (vyžaduje Ampere GPU nebo novější)
    bnb_4bit_use_double_quant=True,  # Mírně zvyšuje úsporu paměti
)

# Načtení modelu s kvantizací
print(f"Načítám model z lokální cesty: {MODEL_PATH}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto",  # Automaticky rozmístí model na dostupné GPU
    trust_remote_code=True,
)
model.config.use_cache = False  # Vypnutí cache je nutné pro trénink s LoRA
model.config.pretraining_tp = 1

# Načtení tokenizeru
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
# Mistral nemá pad_token, nastavíme ho na end-of-sequence token. To je standardní postup.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Důležité pro kauzální modely

Načítám model z lokální cesty: /home/luk-hrd/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/2dcff66eac0c01dc50e4c41eea959968232187fe


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.90s/it]


In [5]:
# --- 4. Konfigurace PEFT (LoRA) ---

# Konfigurace LoRA adaptérů
lora_config = LoraConfig(
    r=16,  # Rank: počet trénovatelných parametrů. 64 je silná hodnota.
    lora_alpha=32,  # Škálovací faktor. Obvykle 2*r nebo r/2.
    lora_dropout=0.1,  # Dropout pro regularizaci LoRA vah.
    bias="none",
    task_type="CAUSAL_LM",
    # Cílové moduly specifické pro Mistral-7B. Cílíme na attention vrstvy.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

In [6]:
# --- 5. Nastavení a spuštění tréninku ---

# Vyladěné trénovací argumenty
training_args = TrainingArguments(
    output_dir=NEW_MODEL_NAME,
    num_train_epochs=1,  # 1 epocha je často dostatečná pro fine-tuning, aby se předešlo overfittingu
    per_device_train_batch_size=1,  # Upravte podle VRAM vaší GPU (snižte, pokud dochází paměť)
    gradient_accumulation_steps=8,  # Efektivní batch size bude 4 * 2 = 8. Pomáhá stabilizovat trénink.
    gradient_checkpointing=True, 
    optim="paged_adamw_32bit",  # Paměťově efektivní optimalizátor od bitsandbytes
    save_steps=50,  # Ukládat checkpoint každých 50 kroků
    logging_steps=10,  # Logovat metriky (např. loss) každých 10 kroků
    learning_rate=2e-4,  # Osvědčená hodnota pro LoRA fine-tuning
    weight_decay=0.001,
    bf16=True,  # Použít bfloat16 (pokud máte starší GPU bez podpory bfloat16, změňte na fp16=True)
    max_grad_norm=0.3,  # Prevence explodujících gradientů
    max_steps=-1,  # -1 znamená, že se řídíme počtem epoch
    warmup_ratio=0.03,  # Pomalý náběh learning rate pro stabilitu
    group_by_length=True,  # Seskupuje texty podobné délky, což výrazně zrychluje trénink
    lr_scheduler_type="cosine",  # 'cosine' scheduler často dává lepší výsledky než 'constant'
    report_to="tensorboard", # Můžete vizualizovat loss v TensorBoardu
)

# Vytvoření instance SFTTrainer (Supervised Fine-tuning Trainer)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    formatting_func=format_prompt_with_context,  # Klíčové: používáme naši vylepšenou funkci
    #max_seq_length=1024,  # Maximální délka sekvence. Ořízne delší texty.
    #tokenizer=tokenizer,
    args=training_args,
)

# --- Spuštění tréninku ---
print("Zahajuji fine-tuning...")
trainer.train()
print("Fine-tuning byl úspěšně dokončen.")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Zahajuji fine-tuning...


Step,Training Loss
10,1.108700
20,0.712900
30,0.572200
40,0.510100
50,0.536700
60,0.509300
70,0.482500
80,0.411200
90,0.448500
100,0.435600


Fine-tuning byl úspěšně dokončen.


In [7]:
# --- Uložení finálního modelu ---
# Uloží se pouze LoRA adaptér, nikoli celý model.
final_output_dir = os.path.join(NEW_MODEL_NAME, "final_adapter")
trainer.save_model(final_output_dir)
print(f"Vyladěný model (LoRA adaptér) byl uložen do: {final_output_dir}")

Vyladěný model (LoRA adaptér) byl uložen do: mistral-7b-quantum-instruct-v1/final_adapter
